In [222]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [223]:
import os, io
import pandas as pd
import random

In [224]:
def get_data(file_path) -> str:
    '''
    This function reads the data from text file in Aiken formatand returns it as a string.
    '''
    # ensure the file exists
    if os.path.exists(file_path):
        # open the file and read its contents
        with io.open(file_path, 'r', encoding='utf8') as f:
            data = f.read()
    else:
        RuntimeWarning(f"File {file_path} does not exist.")

    return data



def create_dataframe(data: str) -> pd.DataFrame:
    '''
    This function takes a string of data in Aiken format and returns a pandas dataframe.
    '''
    # create a pandas dataframe from the substrings with columns 'Question', 'A', 'B', 'C', 'D', 'E', 'Correct'
    df = pd.DataFrame(columns=['Question', 'A', 'B', 'C', 'D', 'E', 'Correct'])

    # get the substrings
    substrings = get_substrings(data)

    # iterate over the substrings and extract the question, answers, and correct answer
    for i, substring in enumerate(substrings):

        # extract the question, answers, and correct answer
        question = substring[0]
        answers = substring[1:6]
        correct = substring[6]

        # strip answers of leading and trailing whitespaces
        answers = [answer.strip() for answer in answers]
        correct = correct.strip()

        # strip the answers of 'A)', 'B)', 'C)', 'D)', 'E)'
        answers = [answer[3:] for answer in answers]

        # strip 'ANSWER: ' from the entries in correct column
        correct = correct[8:] 

        # check the column name in the correct column and add '(correct)' to the entry in the respective column
        if correct == 'A':
            answers[0] += ' (correct)'
        elif correct == 'B':
            answers[1] += ' (correct)'
        elif correct == 'C':
            answers[2] += ' (correct)'
        elif correct == 'D':
            answers[3] += ' (correct)'
        elif correct == 'E':
            answers[4] += ' (correct)'
        else:
            raise ValueError(f'Invalid correct answer: {correct}') 

        new_data = {'Question': question, 'A': answers[0], 'B': answers[1], 'C': answers[2], 'D': answers[3], 'E': answers[4], 'Correct': correct}
        new_row = pd.DataFrame(new_data, index=[i])

        df = pd.concat([df, new_row], ignore_index=True)

    return df



def get_substrings(data: str) -> list:
    '''
    This function takes a string of data in Aiken format and returns a list of substrings.
    '''
    # split the text into substrings after every 7th line
    lines = data.split('\n')
    lines = [line for line in lines if line]  # remove empty lines
    n = 7
    substrings = [lines[i:i + n] for i in range(0, len(lines), n)]

    return substrings



def print_substrings(substrings: list):
    '''
    This function prints the substrings.
    '''
    for i, substring in enumerate(substrings):
        print(f"Substring {i+1}:")
        for line in substring:
            print(line)
        print()



def get_random_questions(df: pd.DataFrame, shuffle_answers=True) -> pd.DataFrame:
    '''
    This function takes a pandas dataframe and returns a random row.
    '''
    # get a random sample of n questions
    random_question = df.sample()

    if shuffle_answers:
        # get the answers from the question
        answers = random_question.iloc[0, 1:6]

        # convert the answers to a list
        answers_list = list(answers)

        # randomly shuffle the answers
        random.shuffle(answers_list)

        # update the answers in the question
        random_question.iloc[0, 1:6] = answers_list
    else:
        random_question = random_question

    return random_question



def create_string(random_question: pd.DataFrame) -> str:
    '''
    This function takes a random question and returns it as a string.
    '''

    # get the question
    question = random_question['Question'].values[0]

    # create the string
    str_question = ('\question ' + question + '\n'
      + '\n'
      + '\\begin{choices}' + '\n'
      + '\t' + '\choice ' + random_question['A'].values[0] + '\n'
      + '\t' + '\choice ' + random_question['B'].values[0] + '\n'
      + '\t' + '\choice ' + random_question['C'].values[0] + '\n'
      + '\t' + '\choice ' + random_question['D'].values[0] + '\n'
      + '\t' + '\choice ' + random_question['E'].values[0] + '\n'
      + '\end{choices}' + '\n')
    
    return str_question



def files_to_string(questions_folder: str) -> list:
    '''
    This function takes a folder with files and returns a list of questions as strings.
    '''
    # get number of files in the folder and ignore hidden files
    files_in_folder = [file for file in os.listdir(questions_folder) if not file.startswith('.')]

    # check if there are five files in the folder
    if len(files_in_folder) == 5:
        # create list to store the strings
        str_questions = []
        print(questions_folder)

        # iterate over the files in the folder
        for file in files_in_folder:
            # get the file path
            file_path = os.path.join(questions_folder, file)
            print('Currect file: '+ file_path)

            # read the data from the file
            data = get_data(file_path)

            # create a dataframe from the data
            df = create_dataframe(data)

            # get a random question with shuffled answers
            random_question = get_random_questions(df, shuffle_answers=True)

            # create the string
            str_question = create_string(random_question)

            # append the string to the list
            str_questions.append(str_question)

        return str_questions
    
    elif len(files_in_folder) == 1:
        # create list to store the strings
        str_questions = []
        print(questions_folder)

        # get file name
        file = files_in_folder[0]

        # get the file path
        file_path = os.path.join(questions_folder, file)
        print('Currect file: '+ file_path)

        # read the data from the file
        data = get_data(file_path)

        # create a dataframe from the data
        df = create_dataframe(data)

        # iterate over the files in the folder
        for i in range(5):
            # get a random question with shuffled answers
            random_question = get_random_questions(df, shuffle_answers=True)

            # create the string
            str_question = create_string(random_question)

            # append the string to the list
            str_questions.append(str_question)

        return str_questions

    else:
        print(f"Folder {questions_folder} does not contain five files.")


def remove_solution(testat: str) -> str:
    '''
    This function removes the solution mark from the testat.
    '''
    # split the testat into substrings
    lines = testat.split('\n')

    # strip (correct) from lines
    testat = [line.replace(' (correct)', '') for line in lines]

    # join the lines to a string
    testat = '\n'.join(testat)

    return testat
        


def create_testat(question_folder: str, testat_name: str, output_folder) -> str:
    '''
    This function creates a testat with five questions and saves it as a tex file.
    '''
    str_questions_list = files_to_string(question_folder)
    
    testat = ('\documentclass[10pt]{exam}' + '\n'
            + '\n'
            + '\\usepackage{geometry}' + '\n'
            + '\\geometry{' + '\n'
            + 'a4paper,' + '\n'
            + 'total={185mm,257mm},' + '\n'
            + 'left=10mm,' + '\n'
            + 'top=25mm,' + '\n'
            + 'bottom=10mm' + '\n'
            + '}' + '\n'
            + '\n'
            + '\\begin{document}' + '\n'
            + '\setlength{\\voffset}{-0.5in}' + '\n'
            + '\setlength{\headsep}{5pt}' + '\n'
            + '\n'
            + '\\fbox{\\fbox{\parbox{8cm}{\centering' + '\n'
            + '\\vspace{2mm}' + '\n'
            + testat_name + '\n'
            + '\\vspace{2mm}' + '\n'
            + '}}}' + '\n'
            + '\\hspace{2mm}' + '\n'
            + '\makebox[0.25\\textwidth]{Name:\enspace\hrulefill} \hspace{5mm}' + '\n'
            + '\makebox[0.2\\textwidth]{Datum:\enspace\hrulefill}' + '\n'
            + '\\vspace{4mm}' + '\n'
            + '\n'
            + '\\begin{questions}' + '\n' 
            + '\n' 
            + str_questions_list[0]
            + '\n'
            + '\\vspace{3mm}'
            + str_questions_list[1]
            + '\n'
            + '\\vspace{3mm}'
            + str_questions_list[2]
            + '\n'
            + '\\vspace{3mm}'
            + str_questions_list[3]
            + '\n'
            + '\\vspace{3mm}'
            + str_questions_list[4]
            + '\n'
            + '\\vspace{3mm}'
            + '\\end{questions}' + '\n'
            + '\n'
            + '\\end{document}' + '\n')    
    
    if not os.path.exists(output_folder):
        print(f"Output folder {output_folder} does not exist.")
        print(f"Testat {testat_name} could not be created.")
    else:
        # save the testat as a tex file (version with solution)
        with open(os.path.join(output_folder, f'solution_{testat_name}.tex'), 'w') as f:
            f.write(testat)
        # save the testat as a tex file (version without solution)
        with open(os.path.join(output_folder, f'{testat_name}.tex'), 'w') as f:
            f.write(remove_solution(testat))

        print(f"Testat {testat_name} created successfully.")


In [225]:
exp_names = {
    'A': 'Waermelehre',
    'B': 'Stroemungsmechanik',
    'C': 'Stationaerer Strom',
    'D': 'Zeitabhaengiger Strom',
    'E': 'Diffusion und Osmose',
    'G': 'Geometrische Optik',
    'H': 'Spektralphotometer',
    'J': 'Ultraschall',
    'K': 'Radioaktivitaet'
}

# set questions folder
questions_folder = '../questions/'
output_folder = '../tex/'

In [226]:
#create_testat(questions_folder, testat_name, output_folder)

In [227]:
# get the subfolders in the questions folder
subfolders = os.listdir(questions_folder)
if '.DS_Store' in subfolders:
    subfolders.remove('.DS_Store')

# iterate over subfolders in the questions folder
for subfolder in sorted(subfolders):
    # set the questions folder
    questions_folder = os.path.join(questions_folder, subfolder)
    # set the testat name
    testat_name = f'Testat - Versuch {subfolder} - ' + exp_names[subfolder]
    # set the output folder
    output_folder = os.path.join(output_folder, subfolder)

    print(f'Creating testat for experiment {subfolder} - {exp_names[subfolder]}')
    print(f'Questions folder: {questions_folder}')
    print(f'Testat name: {testat_name}')
    print(f'Output folder: {output_folder}')

    # create the testat
    create_testat(questions_folder, testat_name, output_folder)
  
    # reset the questions folder
    questions_folder = '../questions'
    # reset the output folder
    output_folder = '../tex'

Creating testat for experiment A - Waermelehre
Questions folder: ../questions/A
Testat name: Testat - Versuch A - Waermelehre
Output folder: ../tex/A
../questions/A
Currect file: ../questions/A/quiz-Physik-SS23-PraktPhysMed-Wärmekapazität-20240418-1145.txt
Currect file: ../questions/A/quiz-Physik-SS23-PraktPhysMed-Wärmeenergie-20240418-1145.txt
Currect file: ../questions/A/quiz-Physik-SS23-PraktPhysMed-Temperatur-20240418-1146.txt
Currect file: ../questions/A/quiz-Physik-SS23-PraktPhysMed-Wasser-20240418-1144.txt
Currect file: ../questions/A/quiz-Physik-SS23-PraktPhysMed-Zusammenhang verschiedener Größen-20240418-1144.txt
Testat Testat - Versuch A - Waermelehre created successfully.
Creating testat for experiment B - Stroemungsmechanik
Questions folder: ../questions/B
Testat name: Testat - Versuch B - Stroemungsmechanik
Output folder: ../tex/B
../questions/B
Currect file: ../questions/B/quiz-Physik-SS23-PraktPhysMed-Hagen-Poiseuille-20240418-1143.txt
Currect file: ../questions/B/qu